## Giải thích chi tiết code Trợ lý giọng nói

# 1. Import thư viện và thiết lập


In [1]:
import requests      # Thư viện để gửi HTTP request đến API
import pyttsx3      # Thư viện chuyển văn bản thành giọng nói (Text-to-Speech)
import pyaudio      # Thư viện để ghi âm từ microphone
import vosk         # Thư viện nhận diện giọng nói offline
import json         # Thư viện xử lý dữ liệu JSON
import os           # Thư viện làm việc với hệ điều hành (kiểm tra file)
import sys          # Thư viện hệ thống (thoát chương trình)
import threading    # Thư viện đa luồng (không sử dụng trong code này)
import queue        # Thư viện hàng đợi (không sử dụng trong code này)
from datetime import datetime, timedelta  # Thư viện xử lý ngày tháng

Điều quan trọng cần lưu ý:

Vosk cần model riêng (~100MB) phải tải về và giải nén

PyAudio có thể khó cài trên một số hệ điều hành

Thứ tự import theo chuẩn PEP 8 (standard library trước, third-party sau)

Error handling để kiểm tra dependencies có đầy đủ không

# 2. Khởi tạo class VoiceAssistant

2.1 Thiết lập Text-to-Speech (TTS)

Giải thích: Phần này thiết lập khả năng "nói" của trợ lý. Nó tìm giọng nói tiếng Nga trong hệ thống và thiết lập tốc độ nói vừa phải.

In [2]:
def __init__(self):
    # Khởi tạo engine chuyển văn bản thành giọng nói
    self.tts_engine = pyttsx3.init()
    
    # Thiết lập tốc độ nói (150 từ/phút)
    self.tts_engine.setProperty('rate', 150)
    
    # Lấy danh sách tất cả giọng nói có sẵn trong hệ thống
    voices = self.tts_engine.getProperty('voices')
    
    # Tìm kiếm giọng nói tiếng Nga để sử dụng
    for voice in voices:
        if 'ru' in voice.id.lower() or 'russian' in voice.name.lower():
            self.tts_engine.setProperty('voice', voice.id)
            break

2.2 Thiết lập Speech Recognition (Nhận diện giọng nói)

Giải thích: Vosk là thư viện nhận diện giọng nói hoạt động offline. Model phải được tải về trước và giải nén vào thư mục dự án.

In [4]:
# Đường dẫn đến model Vosk tiếng Nga
self.model_path = "vosk-model-ru-0.42"

# Kiểm tra xem model có tồn tại không
if not os.path.exists(self.model_path):
    print(f"Modell Vosk không tìm thấy: {self.model_path}")
    sys.exit(1)  # Thoát chương trình nếu không có model

# Khởi tạo model và recognizer
self.model = vosk.Model(self.model_path)
self.recognizer = vosk.KaldiRecognizer(self.model, 16000)  # 16000 Hz sample rate

NameError: name 'self' is not defined

2.3 Thiết lập Microphone

In [ ]:
# Khởi tạo PyAudio để ghi âm
self.audio = pyaudio.PyAudio()
self.stream = None  # Stream sẽ được khởi tạo khi cần ghi âm

2.4 Cấu hình API và cache

In [ ]:
# URL cơ sở của API ngày lễ
self.base_url = "https://date.nager.at/api/v2/publicholidays"
self.country = "RU"  # Mã quốc gia mặc định (Nga)
self.year = datetime.now().year  # Năm hiện tại

# Dictionary để lưu cache dữ liệu đã tải
self.holidays_cache = {}

2.5 Mapping lệnh với hàm xử lý

In [ ]:
# Dictionary ánh xạ tên lệnh với hàm xử lý tương ứng
self.commands = {
    'перечислить': self.list_holidays,     # Liệt kê tên ngày lễ
    'сохранить': self.save_holidays,       # Lưu tên vào file
    'даты': self.save_dates,              # Lưu ngày và tên vào file
    'ближайший': self.nearest_holiday,     # Tìm ngày lễ gần nhất
    'количество': self.count_holidays,     # Đếm số lượng ngày lễ
    'страна': self.change_country,         # Đổi quốc gia
    'год': self.change_year,              # Đổi năm
    'помощь': self.show_help,             # Hiển thị trợ giúp
    'выход': self.exit_assistant          # Thoát chương trình
}

# 3. Các hàm cơ bản

3.1 Hàm nói (speak)

Giải thích: Hàm này vừa hiển thị text trên màn hình vừa đọc to bằng giọng nói.

In [ ]:
def speak(self, text):
    """Hàm chuyển văn bản thành giọng nói"""
    print(f"Trợ lý: {text}")        # In ra màn hình
    self.tts_engine.say(text)       # Chuẩn bị đọc
    self.tts_engine.runAndWait()    # Đọc và chờ hoàn thành

3.2 Hàm nghe (listen)

Giải thích: Hàm này ghi âm 5 giây từ microphone, sau đó sử dụng Vosk để chuyển âm thanh thành văn bản.

In [5]:
def listen(self):
    """Hàm ghi âm và nhận diện giọng nói"""
    try:
        # Thiết lập stream ghi âm
        self.stream = self.audio.open(
            format=pyaudio.paInt16,     # 16-bit audio
            channels=1,                 # Mono (1 kênh)
            rate=16000,                # 16kHz sampling rate
            input=True,                # Input stream (ghi âm)
            frames_per_buffer=8000     # Buffer size
        )
        
        print("Đang nghe... (nói sau tiếng beep)")
        self.speak("Đang nghe")
        
        # Ghi âm trong 5 giây
        frames = []
        for _ in range(0, int(16000 / 8000 * 5)):  # 5 giây
            data = self.stream.read(8000)  # Đọc 8000 mẫu
            frames.append(data)           # Thêm vào danh sách
        
        # Ghép tất cả frame thành một audio
        audio_data = b''.join(frames)
        
        # Nhận diện giọng nói
        if self.recognizer.AcceptWaveform(audio_data):
            result = json.loads(self.recognizer.Result())
            return result.get('text', '').lower().strip()
        
        return ""  # Trả về chuỗi rỗng nếu không nhận diện được
        
    except Exception as e:
        print(f"Lỗi nhận diện giọng nói: {e}")
        return ""
    finally:
        # Đóng stream sau khi sử dụng
        if self.stream:
            self.stream.stop_stream()
            self.stream.close()

# 4. Hàm làm việc với API

4.1 Lấy dữ liệu ngày lễ

Giải thích: Hàm này gọi API để lấy danh sách ngày lễ. Nó sử dụng cache để tránh gọi API nhiều lần cho cùng một dữ liệu.

In [ ]:
def get_holidays(self, year=None, country=None):
    """Lấy danh sách ngày lễ từ API"""
    # Sử dụng giá trị mặc định nếu không được truyền vào
    if year is None:
        year = self.year
    if country is None:
        country = self.country
        
    # Tạo key cho cache
    cache_key = f"{country}_{year}"
    
    # Kiểm tra cache trước khi gọi API
    if cache_key in self.holidays_cache:
        return self.holidays_cache[cache_key]
    
    try:
        # Tạo URL API
        url = f"{self.base_url}/{year}/{country}"
        
        # Gửi GET request với timeout 10 giây
        response = requests.get(url, timeout=10)
        
        # Kiểm tra mã trạng thái HTTP
        if response.status_code == 200:
            holidays = response.json()  # Chuyển JSON thành Python object
            
            # Lưu vào cache để lần sau không cần gọi API
            self.holidays_cache[cache_key] = holidays
            return holidays
        else:
            self.speak(f"Lỗi API: {response.status_code}")
            return []
            
    except requests.exceptions.RequestException as e:
        print(f"Lỗi kết nối: {e}")
        self.speak("Không thể kết nối tới server")
        return []

# 5. Các hàm xử lý lệnh

5.1 Liệt kê ngày lễ

In [ ]:
def list_holidays(self):
    """Hiển thị tên các ngày lễ"""
    holidays = self.get_holidays()  # Lấy dữ liệu
    if not holidays:
        self.speak("Không lấy được danh sách ngày lễ")
        return
    
    # Trích xuất tên ngày lễ
    holiday_names = [holiday['name'] for holiday in holidays]
    
    # Giới hạn hiển thị 10 ngày lễ đầu tiên
    if len(holiday_names) > 10:
        self.speak(f"Tìm thấy {len(holiday_names)} ngày lễ. Liệt kê 10 đầu tiên:")
        holiday_names = holiday_names[:10]
    else:
        self.speak(f"Danh sách ngày lễ {self.country} năm {self.year}:")
    
    # In ra màn hình
    for name in holiday_names:
        print(f"- {name}")
    
    # Chỉ đọc tổng số để tiết kiệm thời gian
    self.speak(f"Đã hiển thị danh sách {len(holiday_names)} ngày lễ")

5.2 Lưu tên ngày lễ vào file

In [ ]:
def save_holidays(self):
    """Lưu tên ngày lễ vào file text"""
    holidays = self.get_holidays()
    if not holidays:
        self.speak("Không lấy được danh sách ngày lễ")
        return
    
    # Tạo tên file
    filename = f"holidays_{self.country}_{self.year}.txt"
    
    try:
        # Mở file để ghi (encoding UTF-8 để hỗ trợ tiếng Nga)
        with open(filename, 'w', encoding='utf-8') as f:
            for holiday in holidays:
                # Ghi từng tên ngày lễ trên một dòng
                f.write(f"{holiday['name']}\n")
        
        self.speak(f"Đã lưu danh sách vào file {filename}")
        print(f"File {filename} được tạo thành công")
        
    except Exception as e:
        print(f"Lỗi lưu file: {e}")
        self.speak("Lỗi khi lưu file")

5.3 Lưu ngày và tên vào file

In [ ]:
def save_dates(self):
    """Lưu cả ngày và tên ngày lễ vào file"""
    holidays = self.get_holidays()
    if not holidays:
        self.speak("Không lấy được danh sách ngày lễ")
        return
    
    filename = f"holidays_with_dates_{self.country}_{self.year}.txt"
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            for holiday in holidays:
                # Ghi theo format: YYYY-MM-DD - Tên ngày lễ
                f.write(f"{holiday['date']} - {holiday['name']}\n")
        
        self.speak(f"Danh sách với ngày tháng đã lưu vào {filename}")
        print(f"File {filename} được tạo thành công")
        
    except Exception as e:
        print(f"Lỗi lưu file: {e}")
        self.speak("Lỗi khi lưu file")

5.4 Tìm ngày lễ gần nhất

In [ ]:
def nearest_holiday(self):
    """Tìm ngày lễ gần nhất với ngày hiện tại"""
    holidays = self.get_holidays()
    if not holidays:
        self.speak("Không lấy được danh sách ngày lễ")
        return
    
    today = datetime.now().date()  # Lấy ngày hiện tại
    nearest = None                 # Ngày lễ gần nhất
    min_diff = float('inf')       # Khoảng cách nhỏ nhất (ban đầu là vô cực)
    
    # Duyệt qua tất cả ngày lễ
    for holiday in holidays:
        # Chuyển string thành date object
        holiday_date = datetime.strptime(holiday['date'], '%Y-%m-%d').date()
        
        # Tính khoảng cách
        if holiday_date >= today:
            # Ngày lễ chưa qua
            diff = (holiday_date - today).days
        else:
            # Ngày lễ đã qua, tính cho năm sau
            next_year_date = holiday_date.replace(year=today.year + 1)
            diff = (next_year_date - today).days
        
        # Cập nhật ngày lễ gần nhất
        if diff < min_diff:
            min_diff = diff
            nearest = holiday
    
    # Thông báo kết quả
    if nearest:
        holiday_date = datetime.strptime(nearest['date'], '%Y-%m-%d').date()
        if holiday_date >= today:
            date_str = holiday_date.strftime('%d.%m.%Y')
            self.speak(f"Ngày lễ gần nhất: {nearest['name']}, {date_str}")
        else:
            next_year_date = holiday_date.replace(year=today.year + 1)
            date_str = next_year_date.strftime('%d.%m.%Y')
            self.speak(f"Ngày lễ gần nhất: {nearest['name']}, {date_str} năm sau")
    else:
        self.speak("Không tìm thấy ngày lễ gần nhất")

5.5 Đếm số lượng ngày lễ

In [6]:
def count_holidays(self):
    """Đếm tổng số ngày lễ"""
    holidays = self.get_holidays()
    count = len(holidays)  # Đếm số phần tử trong list
    
    if count > 0:
        self.speak(f"Quốc gia {self.country} năm {self.year} có {count} ngày lễ chính thức")
    else:
        self.speak("Không tìm thấy ngày lễ nào")

5.6 Đổi quốc gia

In [ ]:
def change_country(self):
    """Thay đổi mã quốc gia"""
    self.speak("Nhập mã quốc gia, ví dụ: RU cho Nga, US cho Mỹ, GB cho Anh")
    
    # Nhập từ bàn phím (đơn giản hóa thay vì dùng giọng nói)
    new_country = input("Nhập mã quốc gia: ").upper().strip()
    
    # Kiểm tra định dạng (mã quốc gia là 2 ký tự)
    if len(new_country) == 2:
        old_country = self.country
        self.country = new_country
        self.speak(f"Đã đổi từ {old_country} sang {self.country}")
    else:
        self.speak("Mã quốc gia không hợp lệ")

5.7 Đổi năm

In [ ]:
def change_year(self):
    """Thay đổi năm tìm kiếm"""
    self.speak("Nhập năm cần tìm")
    
    try:
        # Nhập và chuyển thành số nguyên
        new_year = int(input("Nhập năm: ").strip())
        
        # Kiểm tra phạm vi hợp lệ
        if 2000 <= new_year <= 2030:
            old_year = self.year
            self.year = new_year
            self.speak(f"Đã đổi từ năm {old_year} sang {self.year}")
        else:
            self.speak("Năm phải trong khoảng 2000-2030")
    except ValueError:
        self.speak("Định dạng năm không đúng")

# 6. Xử lý lệnh chính

6.1 Phân tích lệnh

In [ ]:
def process_command(self, text):
    """Xử lý lệnh được nhận diện"""
    text = text.lower().strip()  # Chuyển thành chữ thường và loại bỏ khoảng trắng
    
    # Tìm lệnh trong văn bản
    recognized_command = None
    for command in self.commands:
        if command in text:  # Kiểm tra xem lệnh có xuất hiện trong text không
            recognized_command = command
            break
    
    # Thực thi lệnh
    if recognized_command:
        print(f"Nhận diện lệnh: {recognized_command}")
        try:
            # Gọi hàm xử lý tương ứng
            self.commands[recognized_command]()
        except Exception as e:
            print(f"Lỗi khi thực thi: {e}")
            self.speak("Có lỗi khi thực hiện lệnh")
    else:
        print(f"Không nhận diện được lệnh: '{text}'")
        self.speak("Không hiểu lệnh. Nói 'помощь' để xem danh sách lệnh")

6.2 Vòng lặp chính

In [7]:
def run(self):
    """Vòng lặp chính của chương trình"""
    self.show_help()  # Hiển thị hướng dẫn ban đầu
    
    try:
        while True:  # Vòng lặp vô tận
            # Hiển thị trạng thái hiện tại
            print(f"\nCấu hình: quốc gia={self.country}, năm={self.year}")
            print("Nhấn Enter để bắt đầu ghi âm...")
            input()  # Chờ người dùng nhấn Enter
            
            # Cho phép nhập bằng text để test
            print("Hoặc nhập lệnh bằng văn bản:")
            text_input = input("Lệnh: ").strip()
            
            if text_input:
                # Xử lý lệnh văn bản
                self.process_command(text_input)
            else:
                # Xử lý lệnh giọng nói
                recognized_text = self.listen()
                
                if recognized_text:
                    print(f"Nhận diện: {recognized_text}")
                    self.process_command(recognized_text)
                else:
                    self.speak("Không nhận diện được lệnh")
    
    except KeyboardInterrupt:
        # Xử lý khi người dùng nhấn Ctrl+C
        print("\nChương trình bị dừng bởi người dùng")
        self.speak("Chương trình kết thúc")
    except Exception as e:
        # Xử lý lỗi nghiêm trọng
        print(f"Lỗi nghiêm trọng: {e}")
        self.speak("Có lỗi nghiêm trọng xảy ra")
    finally:
        # Dọn dẹp tài nguyên
        if self.stream:
            self.stream.close()
        self.audio.terminate()

# 7. Hàm main

In [8]:
def main():
    """Hàm chính khởi động chương trình"""
    print("Khởi động trợ lý giọng nói...")
    print("Đảm bảo đã cài đặt các thư viện cần thiết:")
    print("pip install requests pyttsx3 pyaudio vosk")
    print("Và tải model Vosk tiếng Nga\n")
    
    try:
        assistant = VoiceAssistant()  # Tạo instance
        assistant.run()              # Chạy chương trình
    except Exception as e:
        print(f"Lỗi khởi động: {e}")

if __name__ == "__main__":
    main()  # Chạy khi file được thực thi trực tiếp

Khởi động trợ lý giọng nói...
Đảm bảo đã cài đặt các thư viện cần thiết:
pip install requests pyttsx3 pyaudio vosk
Và tải model Vosk tiếng Nga

Lỗi khởi động: name 'VoiceAssistant' is not defined


# Tóm tắt luồng hoạt động

1. **Khởi tạo**: Thiết lập TTS, STT, microphone, API config  
2. **Vòng lặp chính**:
   - Chờ người dùng  
   - Ghi âm hoặc nhập text  
   - Nhận diện lệnh  
   - Thực thi lệnh tương ứng  
   - Trả về kết quả  
3. **Xử lý lỗi**: Báo lỗi rõ ràng cho từng trường hợp  
4. **Dọn dẹp**: Giải phóng tài nguyên khi kết thúc  

# Ưu điểm của thiết kế

- **Modular**: Mỗi chức năng là một hàm riêng biệt  
- **Cache**: Tối ưu hóa việc gọi API  
- **Error handling**: Xử lý lỗi toàn diện  
- **Flexible input**: Hỗ trợ cả giọng nói và text  
- **Extensible**: Dễ dàng thêm lệnh mới  

